
AB Test consumer APP walkaround

In [18]:
import pandas as pd

from config import (
    BASELINE_REDEMPTION_RATE,
    TARGET_REDEMPTION_RATE,
    REQUIRED_SAMPLE_PER_VARIANT,
)

from analysis.metrics import compute_variant_metrics
from analysis.test import run_redemption_ztest


In [19]:
import pandas as pd

from config import (
    BASELINE_REDEMPTION_RATE,
    TARGET_REDEMPTION_RATE,
    REQUIRED_SAMPLE_PER_VARIANT,
)

from data.warehouse import get_warehouse_connection #we are using dummy data but this is what we would do if loading data from relational database
from data.db import load_experiment_outcomes #we are using dummy data but this is what we would do if loading data from relational database
from analysis.metrics import compute_variant_metrics
from analysis.test import run_redemption_ztest



In [20]:
#again we are using dummy data but this is what we would do if loading data from relational database
#conn = get_warehouse_connection()
#df = load_experiment_outcomes(conn, "voucher_ui_01")

Loading Data (Dummy)

In [21]:

data = {
    "user_id": range(1, 2191),
    "variant": ["control"] * 1095 + ["treatment"] * 1095,
    "redeemed": (
        ([1] * 219 + [0] * 876) +      # Control: 20% of 1,095
        ([1] * 274 + [0] * 821)        # Treatment: ~25% of 1,095
    )
}


df = pd.DataFrame(data)



In [22]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 2190 entries, 0 to 2189
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   user_id   2190 non-null   int64
 1   variant   2190 non-null   str  
 2   redeemed  2190 non-null   int64
dtypes: int64(2), str(1)
memory usage: 51.5 KB


Metrics

In [23]:
summary = compute_variant_metrics(df)
summary


,variant,users,redeemers,redemption_rate
0,control,1095,219,0.200000
1,treatment,1095,274,0.250228


Sample Size Check

In [24]:
summary[["variant", "users"]]


,variant,users
0,control,1095
1,treatment,1095


Test

In [25]:
test_results = run_redemption_ztest(summary)
test_results


{'z_stat': np.float64(-2.8139758512197663),
 'p_value': np.float64(0.004893290414284265)}

In [26]:
summary["uplift_vs_control"] = (
    summary["redemption_rate"] /
    summary.loc[summary["variant"] == "control", "redemption_rate"].values[0]
    - 1
)

summary


,variant,users,redeemers,redemption_rate,uplift_vs_control
0,control,1095,219,0.200000,0.000000
1,treatment,1095,274,0.250228,0.251142
